In [ ]:
%pip install torch torchvision pillow scikit-image lpips matplotlib 


  Using cached models-0.9.3.tar.gz (16 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [8 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 35, in <module>
        File "C:\Users\blobf.DESKTOP-IUEL8R6\AppData\Local\Temp\pip-install-8q3wi8ft\models_0d20757c3ae2494c8a442aa4f67535a7\setup.py", line 25, in <module>
          import models
        File "C:\Users\blobf.DESKTOP-IUEL8R6\AppData\Local\Temp\pip-install-8q3wi8ft\models_0d20757c3ae2494c8a442aa4f67535a7\models\__init__.py", line 23, in <module>
          from base import *
      ModuleNotFoundError: No module named 'base'
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, n

Imports

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from torchvision import transforms
from PIL import Image
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
import glob
import os
import math
import random
import numpy as np
import torch
import torch.nn as nn
from scipy.ndimage import laplace, sobel
from torch.utils.data import DataLoader, Dataset
from PIL import Image, ImageFilter
from torchvision import transforms
from torchvision.transforms import Resize, Compose, ToTensor, Normalize
import matplotlib.pyplot as plt

dtype = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Running on device: {device}")

Get LR dataset and HR dataset (for ground truths)

In [ ]:
class DIV2KDataset(Dataset):
    def __init__(self, hr_dir, lr_dir):
        self.hr_paths = sorted(glob.glob(os.path.join(hr_dir, '*.png')))
        self.lr_paths = sorted(glob.glob(os.path.join(lr_dir, '*.png')))

        self.transform = transforms.ToTensor() # PIL to Tensor
    
    def __len__(self): # dataloader needs access to length
        return len(self.hr_paths)

    def __getitem__(self, index): # dataloader needs access to dataset items by index
        lr_img = self.transform(Image.open(self.lr_paths[index])) # DIV2K is RGB images
        hr_img = self.transform(Image.open(self.hr_paths[index]))
        return lr_img, hr_img

BASE_DIR = Path.cwd()
DATA_DIR = BASE_DIR / 'data'

train_dataset = DIV2KDataset(
    hr_dir=str(DATA_DIR / 'DIV2K_train_HR'),
    lr_dir=str(DATA_DIR / 'DIV2K_train_LR_x8')
)

val_dataset = DIV2KDataset(
    hr_dir=str(DATA_DIR / 'DIV2K_valid_HR'),
    lr_dir=str(DATA_DIR / 'DIV2K_valid_LR_x8')
)

Get data - 1 randomly selected image (LR as x0 and corresponding HR for PSNR)

In [ ]:
# select a random index
index = random.randint(0, len(train_dataset) - 1)
print(f"Selecting image index: {index} from dataset")

# get the LR and HR images at that index
img_LR_tensor, img_HR_tensor = train_dataset[index]

# convert from [C, H,W] to [1, C, H, W] and move to GPU
img_LR_var = img_LR_tensor.unsqueeze(0).to(device)
img_HR_var = img_HR_tensor.unsqueeze(0).to(device)

print(f"HR Image Shape: {img_HR_var.shape}")
print(f"LR Input Shape: {img_LR_var.shape}")

SIREN layer

In [ ]:
class SineLayer(nn.Module):
    """ Linear layer followed by the sine activation

    If `is_first == True`, then it represents the first layer of the network.
    In this case, omega_0 is a frequency factor, which simply multiplies the activations before the nonlinearity.
    Note that it influences the initialization scheme.

    If `is_first == False`, then the weights will be divided by omega_0 so as to keep the magnitude of activations constant,
    but boost gradients to the weight matrix.
    """

    def __init__(self, in_features, out_features, bias=True, is_first=False, omega_0=30):
        super().__init__()
        self.omega_0 = omega_0
        self.is_first = is_first

        self.in_features = in_features
        self.linear = nn.Linear(in_features, out_features, bias=bias)
        self.init_weights()

    # initialize weights uniformly
    def init_weights(self):
        with torch.no_grad():
            if self.is_first:
                self.linear.weight.uniform_(-1 / self.in_features,
                                             1 / self.in_features)
            else:
                self.linear.weight.uniform_(-np.sqrt(6 / self.in_features) / self.omega_0,
                                             np.sqrt(6 / self.in_features) / self.omega_0)

    def forward(self, input):
        # 1. pass input through linear layer (self.linear layer performs the linear transformation on the input)
        x = self.linear(input)

        # 2. scale the output of the linear transformation by the frequency factor
        x = x * self.omega_0

        # 3. apply sine activation
        x = torch.sin(x)

        return x

SIREN network definition

In [ ]:
class Siren(nn.Module):
    """ SIREN architecture """

    def __init__(self, in_features, out_features, hidden_features=256, hidden_layers=3, outermost_linear=False,
                 first_omega_0=30, hidden_omega_0=30.):
        super().__init__()

        self.net = []
        self.net.append(SineLayer(in_features, hidden_features,
                                  is_first=True, omega_0=first_omega_0))

        for i in range(hidden_layers):
            self.net.append(SineLayer(hidden_features, hidden_features,
                                      is_first=False, omega_0=hidden_omega_0))

        if outermost_linear:
            final_linear = nn.Linear(hidden_features, out_features)
            with torch.no_grad():
                final_linear.weight.uniform_(-np.sqrt(6 / hidden_features) / hidden_omega_0,
                                              np.sqrt(6 / hidden_features) / hidden_omega_0)
            self.net.append(final_linear)
        else:
            self.net.append(SineLayer(hidden_features, out_features,
                                      is_first=False, omega_0=hidden_omega_0))

        self.net = nn.Sequential(*self.net) # sequential wrapper of SineLayer and Linear

    def forward(self, coords):
        coords = coords.clone().detach().requires_grad_(True)
        output = self.net(coords)
        return output, coords

Define helper functions to create 2d coordinate grid and normalise images to [-1, 1] (i.e. mean and std. of 0.5)

In [ ]:
def get_mgrid(sidelen1,sidelen2, dim=2):
    '''Generates a flattened grid of (x,y,...) coordinates in a range of -1 to 1.
    sidelen: int
    dim: int'''

    if sidelen1 >= sidelen2:
      # use sidelen1 steps to generate the grid
      tensors = tuple(dim * [torch.linspace(-1, 1, steps = sidelen1)])
      mgrid = torch.stack(torch.meshgrid(*tensors), dim = -1)
      # crop it along one axis to fit sidelen2
      minor = int((sidelen1 - sidelen2)/2)
      mgrid = mgrid[:,minor:sidelen2 + minor]

    if sidelen1 < sidelen2:
      tensors = tuple(dim * [torch.linspace(-1, 1, steps = sidelen2)])
      mgrid = torch.stack(torch.meshgrid(*tensors), dim = -1)

      minor = int((sidelen2 - sidelen1)/2)
      mgrid = mgrid[minor:sidelen1 + minor,:]

    # flatten the gird
    mgrid = mgrid.reshape(-1, dim)

    return mgrid

def image_to_tensor(img):
    transform = Compose([
        ToTensor(),
        Normalize(torch.Tensor([0.5]), torch.Tensor([0.5]))
    ])
    img = transform(img)
    return img


Format data

In [ ]:

# 1 get dimensions
_, c, lr_h, lr_w = img_LR_var.shape # lr image is [1, 3, H, W]
_, _, hr_h, hr_w = img_HR_var.shape

# 2 create coordinate grids (to use as the inputs)
lr_coords = get_mgrid(lr_h, lr_w).to(device) # Inputs for training
hr_coords = get_mgrid(hr_h, hr_w).to(device) # Inputs for super-resolution (inference)

# 3 format pixel values (the targets for the network)

# flatten image pixels to match list of coordinates.
# permute to (H, W, C), then flatten to (N, 3) - i.e. a list of RGB values for each pixel coordinate
lr_pixels = img_LR_var.squeeze(0).permute(1, 2, 0).reshape(-1, 3)
hr_pixels = img_HR_var.squeeze(0).permute(1, 2, 0).reshape(-1, 3)

# normalise from [0, 1] to [-1, 1]
lr_pixels = (lr_pixels - 0.5) / 0.5
hr_pixels = (hr_pixels - 0.5) / 0.5

print(f"Training Input (Coords): {lr_coords.shape}") # Should be (N_lr, 2)
print(f"Training Target (Pixels): {lr_pixels.shape}") # Should be (N_lr, 3)

Initialise and train model

In [ ]:
# initialise model
model = Siren(in_features=2, out_features=3, hidden_features=256, 
              hidden_layers=3, outermost_linear=True).to(device)

# adam as optimiser
optim_inr = torch.optim.Adam(lr=1e-4, params=model.parameters())

# training loop
steps = 2000 # can be increased
print("Starting training...")

for step in range(steps):
    # 1 input LR coordinates to model
    model_output, _ = model(lr_coords)
    
    # 2 compare output to LR Pixels
    loss = ((model_output - lr_pixels)**2).mean() # MSE Loss

    # 3 optimise
    optim_inr.zero_grad()
    loss.backward()
    optim_inr.step()

    if step % 200 == 0:
        print(f"Step {step}, Loss: {loss.item():.6f}")

    # --- VISUALIZATION BLOCK --- #####################################
    if step % print_every == 0:
        # Switch to evaluation mode (optional but good practice)
        model.eval()
        
        with torch.no_grad():
            # A. Ask model to predict High Res image
            sr_output, _ = model(hr_coords)
            
            # B. Reshape result: (N, 3) -> (H, W, 3)
            # We use hr_h and hr_w variables we defined earlier
            pred_img = sr_output.view(hr_h, hr_w, 3).cpu().numpy()
            
            # C. Un-normalize: [-1, 1] -> [0, 1]
            pred_img = (pred_img + 1) / 2
            pred_img = np.clip(pred_img, 0, 1)

            # D. Plotting
            clear_output(wait=True) # Clears the previous image
            
            plt.figure(figsize=(12, 5))
            
            # Plot 1: The current Super-Resolution output
            plt.subplot(1, 2, 1)
            plt.title(f"Step {step} | Loss: {loss.item():.5f}")
            plt.imshow(pred_img)
            plt.axis('off')
            
            # Plot 2: The Ground Truth (for comparison)
            plt.subplot(1, 2, 2)
            plt.title("Ground Truth (High Res)")
            # Convert the original HR tensor to numpy for display
            gt_display = img_HR_var.squeeze(0).permute(1, 2, 0).cpu().numpy()
            plt.imshow(gt_display)
            plt.axis('off')
            
            plt.show()
        
        # Switch back to training mode
        model.train()

        #####################################################################

print("Training Complete.")

Inference and PSNR

In [ ]:
print("Upscaling...")

# query with HR coordinates
with torch.no_grad():
    sr_output, _ = model(hr_coords)

# reshape back to image format
sr_img = sr_output.view(hr_h, hr_w, 3).cpu().numpy()

# un-normalise: [-1, 1] back to [0, 1]
sr_img = (sr_img + 1) / 2
sr_img = np.clip(sr_img, 0, 1)

# get ground truth
gt_img = img_HR_var.squeeze(0).permute(1, 2, 0).cpu().numpy()

# PSNR
mse = np.mean((gt_img - sr_img) ** 2)
sr_psnr = 20 * np.log10(1.0 / np.sqrt(mse))
print(f"Super-Resolution PSNR: {sr_psnr:.2f} dB")

# visualise
plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1); plt.title("LR Input"); plt.imshow(img_LR_var.squeeze(0).permute(1, 2, 0).cpu().numpy())
plt.subplot(1, 3, 2); plt.title(f"INR Output ({sr_psnr:.2f} dB)"); plt.imshow(sr_img)
plt.subplot(1, 3, 3); plt.title("Ground Truth"); plt.imshow(gt_img)
plt.show()